In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip3 install unidecode

     |████████████████████████████████| 245kB 2.8MB/s 


In [3]:
import sys
sys.path.append("your-path/Machine Translation")


In [4]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Input, Dropout, GRU, Embedding, Dense
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from unidecode import unidecode
import re
import string
from pickle import dump, load
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from collections import Counter
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import model_from_json
from os import path
from DataGenerator import DataGenerator

Using TensorFlow backend.


# Data preprocessing

In [5]:
class NMT:

	def __init__(self, dataset_english_path, dataset_french_path, clean_en_path, clean_fr_path, encoder_input_path, 
	             decoder_input_path, decoder_output_path, vocab_path, max_sentence_length):
			
		self.dataset_english_path = dataset_english_path
		self.dataset_french_path = dataset_french_path
		self.clean_en_path = clean_en_path
		self.clean_fr_path = clean_fr_path
		self.encoder_input_path = encoder_input_path
		self.decoder_input_path = decoder_input_path
		self.decoder_output_path = decoder_output_path
		self.max_sentence_length = max_sentence_length
		self.vocab_path = vocab_path

		if path.exists(self.clean_en_path) and  path.exists(self.clean_fr_path):
				self.dataset_english = self.load_prep_sentences(self.clean_en_path)
				self.dataset_french = self.load_prep_sentences(self.clean_fr_path)
		else:
				self.dataset_english = self.load_doc(self.dataset_english_path)
				self.dataset_french = self.load_doc(self.dataset_french_path)
				self.save_prep_sentences(self.dataset_english, self.clean_en_path)
				self.save_prep_sentences(self.dataset_french, self.clean_fr_path)



		if path.exists(self.encoder_input_path) and path.exists(self.decoder_input_path) and path.exists(self.decoder_output_path):
			
			print("Loading preprocessed datasets...\n")
	 
			self.encoder_input = self.load_prep_sentences(self.encoder_input_path)
			self.decoder_input = self.load_prep_sentences(self.decoder_input_path)
			self.decoder_output = self.load_prep_sentences(self.decoder_output_path)
			self.vocab_dict = self.load_prep_sentences(self.vocab_path)

			print("Loaded succesfully!\n")
		else:

			self.encoder_input, self.decoder_input, self.decoder_output = self.prepare_dataset(self.dataset_english, self.dataset_french,
			                                                                                   keep_all = True)
			self.save_prep_sentences(self.encoder_input,self.encoder_input_path)
			self.save_prep_sentences(self.decoder_input,self.decoder_input_path)
			self.save_prep_sentences(self.decoder_output,self.decoder_output_path)
			self.save_prep_sentences(self.vocab_dict, self.vocab_path)


	
	def prepare_dataset(self, dataset_english, dataset_french, keep_all = True):

		
		sentences_fr = self.dataset_french.copy()
		sentences_eng = self.dataset_english.copy()


		if not keep_all:
			sentences_eng = sentences_eng[:1000]
			sentences_fr = sentences_fr[:1000]
		

		sentences_eng, sentences_fr = self.reduce_sequences(sentences_eng, sentences_fr)
	
		sentences_fr, decoder_input_fr, decoder_output_fr = self.insert_start_stop(sentences_fr)
		fr_vocab_size, vocab_fr, decoder_input_fr, decoder_output_fr = self.tokenize(sentences_fr, decoder_input_fr, decoder_output_fr)
	
		eng_vocab_size, vocab_eng, encoder_eng = self.tokenize(sentences_eng)

		self.vocab_dict = {
				"vocab_fr" : vocab_fr,
				"fr_vocab_size" : fr_vocab_size,
				"vocab_eng" : vocab_eng,
				"eng_vocab_size" : eng_vocab_size
		}


		encoder_padded = self.pad_seq(encoder_eng)
		decoder_input_padded = self.pad_seq(decoder_input_fr)
		decoder_output_padded = self.pad_seq(decoder_output_fr)
	

		return encoder_padded, decoder_input_padded, decoder_output_padded




	def load_doc(self,filename):
		file = open(filename, mode='rt', encoding='utf-8')
		text = file.read()
		file.close()
	
		return self.clean_sentences(text)

	def clean_sentences(self,doc):
		sentences =  doc.strip().split('\n')
		sentences = [re.sub(r"[^\w\s]","",unidecode(sent.lower())) for sent in sentences]
		return sentences


	def save_prep_sentences(self,sentences, filename):
		dump(sentences, open(filename, 'wb'))
		print('Saved: ', filename)


	def load_prep_sentences(self,filename):
		return load(open(filename, 'rb'))
	

	def insert_start_stop(self,target):

			'''Insert "start" string at the beginning of each sequence from decoder_input and "stop"
		 at the end of each sequence from decoder_output. We do this in order to enforce teacher forcing (where
		 the decoder is trained to predict the next word given the previous words of the sentence).
			'''
		decoder_input = []
		decoder_output = []

		for i,st in enumerate(target):
			target[i] = " ".join(("start",st, "stop"))
			decoder_input.append(" ".join(("start",st)))
			decoder_output.append(" ".join((st, "stop")))
		
		return target, decoder_input, decoder_output

	def reduce_vocabulary(self,sequence, min_occurence):
		'''
		Keep only the words that occur at least min_occurence times in 
		eng/fr sentences in order to reduce the vocabulary size (if needed)
		'''
		count_obj = Counter()
	
		for seq in sequence:
			count_obj.update(seq.split())
		
		nr_words_to_keep = 0
		for word, nr_occurences in count_obj.items():
			if nr_occurences >= min_occurence:
				nr_words_to_keep += 1

		
		return nr_words_to_keep

	

	def tokenize(self,list_sentences, *args):
		
		'''Create vocabulary and transform text sentences into number sequences based on the vocabulary'''

		min_occurence = 1
		num_words = self.reduce_vocabulary(list_sentences, min_occurence)


		tok = Tokenizer(num_words = num_words + 1, oov_token='OOV')
		tok.fit_on_texts(list_sentences)
		vocab = tok.word_index

		if len(args) == 0:
			return len(vocab), vocab, tok.texts_to_sequences(list_sentences)

		return len(vocab), vocab, tok.texts_to_sequences(args[0]), tok.texts_to_sequences(args[1])



	def max_len(self,sequences):
		return max([len(s.split()) for s in sequences])



	def reduce_sequences(self, sentences_eng, sentences_fr):
		'''Reduce the number of sequences by imposing a max-length on each eng & fr pair of sentences'''

			sent_eng = []
			sent_fr = []

			for s_en, s_fr in zip(sentences_eng, sentences_fr):
				if len(s_en.split()) <= self.max_sentence_length and len(s_fr.split()) <= self.max_sentence_length and \
					len(s_en.split()) > 0 and len(s_fr.split()) > 0:
					sent_eng.append(s_en)
					sent_fr.append(s_fr)
			
			assert self.max_len(sent_eng) <= self.max_sentence_length
			assert self.max_len(sent_fr) <= self.max_sentence_length

			return sent_eng, sent_fr


	def pad_seq(self,sequences):
		return pad_sequences(sequences, padding = 'post')




In [6]:
dataset_english_path = r"your-path/small_vocab_en.txt"
dataset_french_path = r"your-path/small_vocab_fr.txt"
encoder_input_path = r"your-path/encoder_input.plk"
decoder_input_path = r"your-path/decoder_input.plk"
encoder_output_path = r"your-path/decoder_output.plk"
clean_en_path = r"your-path/clean_en_git.plk"
clean_fr_path = r"your-path/clean_fr_git.plk"
vocab_path = r"your-path/vocab.plk"
max_sentence_length = 30



In [ ]:
nmt = NMT(dataset_english_path, dataset_french_path, clean_en_path, clean_fr_path, encoder_input_path, decoder_input_path, 
          encoder_output_path, vocab_path, max_sentence_length)

In [67]:
print("Number of english sentences: {}\n".format(len(nmt.encoder_input)))
print("Number of french sentences: {}\n".format(len(nmt.decoder_input)))
print("The length of english vocabulary: {}\n".format(nmt.vocab_dict['eng_vocab_size']))
print("The length of french vocabulary: {}\n".format(nmt.vocab_dict['fr_vocab_size']))
  

Number of english sentences: 137860

Number of french sentences: 137860

The length of english vocabulary: 200

The length of french vocabulary: 342



In [8]:
reverse_english_dict = dict((i, word) for word, i in nmt.vocab_dict["vocab_eng"].items())
reverse_french_dict = dict((i, word) for word, i in nmt.vocab_dict["vocab_fr"].items())

# Encoder-Decoder Model

## Split the data into training, validation, testing

In [9]:
encoder_train = nmt.encoder_input[:int(len(nmt.encoder_input) * 0.90)]
encoder_validation = nmt.encoder_input[int(len(nmt.encoder_input) * 0.90):]

decoder_input_train = nmt.decoder_input[:int(len(nmt.decoder_input) * 0.90)]
decoder_input_validation = nmt.decoder_input[int(len(nmt.decoder_input) * 0.90):]

decoder_output_train = nmt.decoder_output[:int(len(nmt.decoder_output) * 0.90)]
decoder_output_validation = nmt.decoder_output[int(len(nmt.decoder_output) * 0.90) : ]


In [11]:
n_units = 256

num_encoder_tokens = encoder_train.shape[1]
num_decoder_tokens = decoder_input_train.shape[1]
num_classes = nmt.vocab_dict['fr_vocab_size']

training_generator = DataGenerator(encoder_train, decoder_input_train, decoder_output_train, 1024, num_classes)
validation_generator = DataGenerator(encoder_validation, decoder_input_validation, decoder_output_validation, 1024, num_classes)


# Model with embedding for decoder input

In [ ]:
'''encoder_inputs = Input(shape=(None,), name = "encoder_input")

encoder_embeddings = Embedding(input_dim = nmt.vocab_dict['eng_vocab_size'], output_dim = 200, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(n_units, return_sequences = False, return_state=True, activation='relu')
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embeddings)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,), name = "decoder_input")
decoder_embedding = Embedding(input_dim = nmt.vocab_dict['fr_vocab_size'], output_dim = 200, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(n_units, return_state = True, return_sequences=True, activation='relu')
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

decoder_dense = Dense(num_classes, activation='softmax', name = "decoder_dense")
decoder_logits = decoder_dense(decoder_outputs)


model = Model([encoder_inputs, decoder_inputs], decoder_logits)

model.compile(optimizer=Adam(learning_rate=0.003), loss='categorical_crossentropy', metrics=['accuracy'])'''


# Model without embedding the decoder input

In [12]:
encoder_inputs = Input(shape=(None,), name = "encoder_input")

encoder_embeddings = Embedding(input_dim = nmt.vocab_dict['eng_vocab_size'], output_dim = 200, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(n_units, return_sequences = False, return_state=True, activation='relu')
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embeddings)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,num_classes), name = "decoder_input")
decoder_lstm = LSTM(n_units, return_state = True, return_sequences=True, activation='relu')
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_classes, activation='softmax', name = "decoder_dense")
decoder_logits = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_logits)

model.compile(optimizer=Adam(learning_rate=0.003), loss='categorical_crossentropy', metrics=['accuracy'])


In [13]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    40000       encoder_input[0][0]              
__________________________________________________________________________________________________
decoder_input (InputLayer)      (None, None, 342)    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 467968      embedding_1[0][0]                
____________________________________________________________________________________________

In [24]:
history = model.fit_generator(generator = training_generator, validation_data = validation_generator,epochs = 5)

Epoch 1/5
121/121 [==============================] - 217s 2s/step - loss: 0.0263 - accuracy: 0.9924 - val_loss: 0.0289 - val_accuracy: 0.9927
Epoch 2/5
121/121 [==============================] - 218s 2s/step - loss: 0.0219 - accuracy: 0.9936 - val_loss: 0.0234 - val_accuracy: 0.9933
Epoch 3/5
121/121 [==============================] - 218s 2s/step - loss: 0.0190 - accuracy: 0.9945 - val_loss: 0.0181 - val_accuracy: 0.9938
Epoch 4/5
121/121 [==============================] - 217s 2s/step - loss: 0.0172 - accuracy: 0.9949 - val_loss: 0.0210 - val_accuracy: 0.9944
Epoch 5/5
121/121 [==============================] - 216s 2s/step - loss: 0.0162 - accuracy: 0.9953 - val_loss: 0.0179 - val_accuracy: 0.9944


# Save model

In [25]:
model_json = model.to_json()
with open("your-path/model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("your-path/model.h5")


# Load model

In [55]:
json_file = open('your-path/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("your-path/Machine Translation/model.h5")

In [56]:
loaded_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    40000       encoder_input[0][0]              
__________________________________________________________________________________________________
decoder_input (InputLayer)      (None, None, 342)    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 467968      embedding_1[0][0]                
____________________________________________________________________________________________

# Inference step
<ul>
<li>Retrieve the output of the layers from the loaded model to be used for inference models</li>
</ul>

In [57]:
encoder_inputs = loaded_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = loaded_model.layers[3].output #lstm1


In [58]:
encoder_states = [state_h_enc, state_c_enc]
decoder_inputs = loaded_model.input[1]
decoder_lstm = loaded_model.layers[4] #lstm2
decoder_dense = loaded_model.layers[5]

In [59]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(n_units,))
decoder_state_input_c = Input(shape=(n_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

In [60]:
decoder_states = [state_h_dec, state_c_dec]
decoder_logits = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_logits] + decoder_states)

## Decode sequences / Make predictions

In [62]:
def decode_sequence(input_seq):

    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1,1,num_classes))

    target_seq[0,0,nmt.vocab_dict["vocab_fr"]['start']] = 1



    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
       



        sampled_token_index = np.argmax(output_tokens[0,0, :])
        sampled_word = reverse_french_dict[sampled_token_index]
        decoded_sentence += sampled_word + ' '


        if (sampled_word == 'stop' or len(decoded_sentence) > 30):
            stop_condition = True

        target_seq = np.zeros((1,1,num_classes))
        target_seq[0,0,sampled_token_index] = 1

        states_value = [h, c]

    return decoded_sentence

<ul>
<li>Function to convert number sequences back into their corresponding strings</li>
</ul>

In [63]:
def index_to_word(seq, rev):
  dec = ' '
  for idx in seq[seq != 0]:
    dec += rev[idx] + ' '
  
  return dec


<ul>
<li>Levenshtein distance between two words is the minimum number of single-character edits (insertions, deletions or substitutions) required to change one word into the other.</li>
</ul>

In [84]:
def levenshtein(a,b):
    m = np.zeros((len(a)+1, len(b)+1))

    for i in range(1,m.shape[0]):
      m[i,0] = m[i-1,0] + 1

    for i in range(1,m.shape[1]):
      m[0,i] = m[0,i-1] + 1
    
    for i in range(1,m.shape[0]):
      for j in range(1, m.shape[1]):
        if a[i-1] != b[j-1]:
          m[i,j] = min(m[i-1,j], m[i,j-1], m[i-1,j-1]) + 1
        else:
          m[i,j] = m[i-1,j-1]
    
    return m[-1,-1]

## Validation set predictions
As we can see below, since only a limited number of sentences (on validation data) have a levenshtein distance $\leq 4$, we assume that our model is under-fitting.

In [52]:

for i in range(len(encoder_validation)):
  inp = index_to_word(encoder_validation[i], reverse_english_dict)
  gt = index_to_word(decoder_input_validation[i], reverse_french_dict)[6:]
  pred = decode_sequence(encoder_validation[i])[:-5]
  if levenshtein(gt.split(), pred.split()) / len(gt.split()) <= 0.4:
    print('\nInput: {}\n'.format(inp))
    print('Ground truth: {}\n'.format(gt))
    print("Predicted: {}\n".format(pred))
    print("-" * 20)



Input:  i dislike oranges 

Ground truth:  je naime les oranges 

Predicted: je naime les mangues 

--------------------

Input:  i dislike grapes 

Ground truth:  je naime pas les raisins 

Predicted: je naime les mangues 

--------------------

Input:  her most loved animal is this bear 

Ground truth:  son animal le plus aime est cet ours 

Predicted: son fruit le plus aime est la chaux 

--------------------

Input:  my most loved animal is this bear 

Ground truth:  mon animal le plus aime est cet ours 

Predicted: mon fruit le plus aime est lorange 

--------------------

Input:  my most loved animal is the horse 

Ground truth:  mon animal le plus aime est le cheval 

Predicted: mon fruit le plus aime est lorange 

--------------------

Input:  her most loved animal is this lion 

Ground truth:  son animal le plus aime est ce lion 

Predicted: son fruit le plus aime est la chaux 

--------------------

Input:  i dislike strawberries 

Ground truth:  je naime pas les fraises 

P

## Training set predictions
To check the assumption of under-fitting i have also printed the predictions on the training data, and indeed we confirm it since, again, a limited number of the predictions have a decent levenshtein distance with respect to the original sentences.

In [54]:
for i in range(len(encoder_train)):
  inp = index_to_word(encoder_train[i], reverse_english_dict)
  gt = index_to_word(decoder_input_train[i], reverse_french_dict)[6:]
  pred = decode_sequence(encoder_train[i])[:-5]
  if levenshtein(gt.split(), pred.split()) / len(gt.split()) <= 0.3:
    print('\nInput: {}\n'.format(inp))
    print('Ground truth: {}\n'.format(gt))
    print("Predicted: {}\n".format(pred))
    print("-" * 20)


Input:  her most loved fruit is the grape 

Ground truth:  son fruit le plus aime est le raisin 

Predicted: son fruit le plus aime est la chaux 

--------------------

Input:  her most loved fruit is the peach 

Ground truth:  son fruit le plus aime est la peche 

Predicted: son fruit le plus aime est la chaux 

--------------------

Input:  they like lemons 

Ground truth:  ils aiment les citrons 

Predicted: ils aiment les citrons 

--------------------

Input:  my most loved fruit is the grape 

Ground truth:  mon fruit le plus aime est le raisin 

Predicted: mon fruit le plus aime est lorange 

--------------------

Input:  her most loved fruit is the mango 

Ground truth:  son fruit le plus aime est la mangue 

Predicted: son fruit le plus aime est la chaux 

--------------------

Input:  we like lemons 

Ground truth:  nous aimons les citrons 

Predicted: nous aimons les citrons 

--------------------

Input:  her most loved fruit is the lime 

Ground truth:  son fruit le plus 

# Conclusion
Since our model is under-fitting, getting more data would not be too helpful, instead adding more layers to the network is a possible solution combined with regularization methods (like Dropout) to prevent over-fitting. The key takeaway from this project is that even though i've used a simple model (with a single LSTM layer), the encoder-decoder architecture could be succesfully used for a task like Neural Machine Translation.